In [6]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.9 MB/s eta 0:00:00


In [8]:
from huggingface_hub import login
login()

In [11]:
import torch
import pandas as pd
import bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:
data = pd.read_csv('test_en.txt', sep='\t')

In [3]:
data

,Sentence,Style
0,this is a free country that protectsamendment ...,toxic
1,generous seperation packages my ass !,toxic
2,"obama has been a total failure , and now looks...",toxic
3,voting for her if the examiner and mitch the e...,toxic
4,the message is pbot doesnt give a shit about y...,toxic
...,...,...
3127,We already have a lot of indians coming here d...,neutral
3128,"If they used it, they are responsible for it, ...",neutral
3129,i don 't know why people are downvoting me .,neutral
3130,They mistreat you.,neutral


In [4]:
sentences = data['Sentence'].values.tolist()
sentences[0]

'this is a free country that protectsamendment rights from little commies like you .'

In [10]:
model_name = 'meta-llama/Llama-3.1-8B'

In [14]:
bitsandbytes_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bitsandbytes_config,
                                             device_map='cuda:0')

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [16]:
sample_sentence = sentences[0]
sample_sentence

'this is a free country that protectsamendment rights from little commies like you .'

In [17]:
prompt = f'Classify the following text into either \'TOXIC\' or \'NEUTRAL\': {sample_sentence}'
prompt

"Classify the following text into either 'TOXIC' or 'NEUTRAL': this is a free country that protectsamendment rights from little commies like you ."

In [20]:
tokens = tokenizer(prompt, return_tensors='pt').to('cuda:0')
tokens

{'input_ids': tensor([[128000,   1999,   1463,    279,   2768,   1495,   1139,   3060,    364,
           5319,     55,   1341,      6,    477,    364,   4031,   1406,  52831,
           1232,    420,    374,    264,   1949,   3224,    430,  36236,    309,
          14988,   3268,    505,   2697,   1081,    552,   1093,    499,    662]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [21]:
output_ids = model.generate(tokens.input_ids)
output_ids

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[128000,   1999,   1463,    279,   2768,   1495,   1139,   3060,    364,
           5319,     55,   1341,      6,    477,    364,   4031,   1406,  52831,
           1232,    420,    374,    264,   1949,   3224,    430,  36236,    309,
          14988,   3268,    505,   2697,   1081,    552,   1093,    499,    662,
           1442,    499,   2733,    430,    279,   1495,    374,    364,   5319,
             55,   1341,      6,   1243,    499,    649,   4299,    279,    364,
          10577,      6]], device='cuda:0')

In [22]:
tokenizer.decode(output_ids[0], skip_special_tokens=True)

"Classify the following text into either 'TOXIC' or 'NEUTRAL': this is a free country that protectsamendment rights from little commies like you. If you feel that the text is 'TOXIC' then you can click the 'Report'"